In [ ]:
import os
import time
import hashlib
import smtplib
from email.mime.text import MIMEText

# Directory to monitor for changes
directory_to_monitor = "/path/to/your/directory"

# Email settings
smtp_server = "your_smtp_server"
smtp_port = 587
sender_email = "sender_email@example.com"
sender_password = "sender_password"
receiver_email = "receiver_email@example.com"

# Hash table to store file hashes
file_hashes = {}

def send_email(subject, body):
    message = MIMEText(body)
    message['Subject'] = subject
    message['From'] = sender_email
    message['To'] = receiver_email

    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, message.as_string())

def calculate_file_hash(file_path):
    hasher = hashlib.sha256()
    with open(file_path, 'rb') as file:
        buffer = file.read(65536)
        while len(buffer) > 0:
            hasher.update(buffer)
            buffer = file.read(65536)
    return hasher.hexdigest()

def monitor_directory():
    print("Monitoring directory for changes...")
    while True:
        for root, dirs, files in os.walk(directory_to_monitor):
            for file in files:
                file_path = os.path.join(root, file)
                file_hash = calculate_file_hash(file_path)

                if file_path in file_hashes:
                    if file_hashes[file_path] != file_hash:
                        print(f"[WARNING] File change detected: {file_path}")
                        send_email("Suspicious file change detected", f"File change detected: {file_path}")
                else:
                    print(f"[INFO] New file detected: {file_path}")
                    send_email("New file detected", f"New file detected: {file_path}")

                file_hashes[file_path] = file_hash

        time.sleep(60)  # Adjust the interval as per your requirement

monitor_directory()
